In [4]:
from model.FieldingModel import FieldingModel


In [5]:
league =  'BZ'
season_start = 2016
season_end = 2064
ip_limit = 10
epochs = 10000
positions = [9,8,7,6,5,4,3,2]
ratings_type = "Mixed"

In [6]:

    
for position in positions:
    print(f"\r\nTraining model for position {position}")
    fieldingModel = FieldingModel(league, season_start, season_end, position, ratings_type)
    fieldingModel.load_data(ip_limit)

    epoch_count, loss = fieldingModel.train(epochs)
    print(f"Epoch [{epoch_count}/{epochs}], Loss: {loss:.7f}")

    test_loss = fieldingModel.evaluate()
    print(f"Test Mean Squared Error: {test_loss:.7f}")

    #feature_importance = fieldingModel.feature_importance()
    #print(feature_importance)

    fieldingModel.save_model()

    predictions = fieldingModel.predict(season_end+1, ip_limit)
    predictions.sort_values(by='Predictions', ascending=False, inplace=True)
    print(predictions.head(10))


Training model for position 9
Empty DataFrame
Columns: [WT, SPE, OF RNG, OF ARM, OF ERR, RF, ID, runsPAdjSeason]
Index: []
(5457, 8)
Epoch [2000/10000], Best Loss: 24.4412003
Epoch [4000/10000], Best Loss: 23.4255810
Epoch [6000/10000], Best Loss: 22.9273415
Epoch [8000/10000], Best Loss: 22.4770031
Epoch [10000/10000], Best Loss: 22.1637917
Epoch [9999/10000], Loss: 22.2085896
Test Mean Squared Error: 25.7612591
      WT  SPE  OF RNG  OF ARM  OF ERR  RF     ID  Predictions
885  201    4       8       8       7   9  66260    22.429121
799  204    5       7       8       5   8  63218    19.492954
794  205    6       7       7       7   7  63188    18.271803
836  204    5       7       6       4   7  63916    18.251251
312  201    8       8      10      10  10  56617    17.404209
827  199    5       8       7       6   7  63678    17.213198
758  200    8       7       8       5   8  62217    16.744690
719  194    8       7      10       8   9  61845    16.509468
618  199    7       7   